In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 

from transformers import BertForSequenceClassification, BertTokenizer, BertForMaskedLM

from simpletransformers.language_modeling import LanguageModelingModel

import spacy

from sklearn.svm import LinearSVC
from sklearn.metrics.pairwise import cosine_similarity, paired_euclidean_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import normalize, StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import itertools
from collections import Counter

from tqdm import tqdm
import torch

import networkx as nx

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from functools import partial

import pickle

from collections import deque

from torchvision import models
import torch.nn.functional as F
from torch import nn

from sklearn.utils import resample

import fasttext


from typing import List
from pathlib import Path
from joblib import dump, load

import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/home/roshansk/YelpAnalysis/')
from utils import *



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
nlp = spacy.load('/data2/link10/models/fasttext/en_fasttext_crawl')
# nlp = spacy.load('/data2/link10/models/fasttext/en_fasttext_crawl_subword/')

sys.path.insert(0,'/data2/Datasets/')
from preprocess import *

dataFolder = '/data2/Datasets/Raw'

### Load Data & Model Training

In [5]:
trainDf, devDf, testDf = splitData(getData(dataFolder, 'empathy'))

trainData = generateFastTextData_Spacy(trainDf, nlp, textVariable = 'text')

testData = generateFastTextData_Spacy(testDf, nlp, textVariable = 'text')

model = trainSVM(trainData, testData, trainDf, testDf)

lexicon = generateLexicon_SVM(model,trainDf, nlp)

In [24]:
lexicon.head()

,word,score,wordCount,SVM_Rank
5346,circuses,41.932019,1,0
7783,graham,32.510310,1,1
4070,fernandez,32.277996,2,2
3278,shipwreck,29.755081,2,3
7312,job.i,26.816461,1,4


In [25]:
lexiconWords, lexiconMap = getLexicon(df = lexicon)


### Experiments

In [12]:
dataList = ['dialog','amazon_finefood_subset','amazon_toys_subset','yelp_subset','empathy','nrc_joy']
for data in dataList:
    results = testSVM(model, data,lexiconWords, lexiconMap, nlp, dataFolder)
    

dialog , 0.838 , 0.908 , 0.838 , 0.907
amazon_finefood_subset , 0.869 , 0.928 , 0.872 , 0.929
amazon_toys_subset , 0.94 , 0.969 , 0.935 , 0.966
yelp_subset , 0.884 , 0.924 , 0.879 , 0.921
empathy , 0.505 , 0.614 , 0.548 , 0.621
nrc_joy , 0.635 , 0.366 , 0.629 , 0.333


In [26]:
dataList = ['songs','emobank']
for data in dataList:
    results = testSVM(model, data,lexiconWords, lexiconMap, nlp, dataFolder)

songs , 0.7744322344322344 , 0.8711197856579798 , 0.777 , 0.874
emobank , 0.37616689332460285 , 0.5391576140313964 , 0.626 , 0.013
